# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Boston Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import the Boston data set using `load_boston()`. We won't bother to preprocess the data in this lab. If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()
boston = load_boston()

Create a baseline model which includes all the variables in the Boston housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [17]:
## code here
print(boston.keys())
print(boston.feature_names)
y= pd.DataFrame(boston.target,columns=['target'])
df = pd.DataFrame(boston.data,columns = boston.feature_names)
all_data = pd.concat([y,df],axis=1)

# calculate the mean of Rsquared from multiple models obtained through K-Fold process
# partition of df and y in 10 equal parts, creates 10 models using each time 1 out of the 10 parts as testing set
# and the other 9 out of 10 as training set, and changing everytime.
crossvalidation = KFold(n_splits=10,shuffle=True,random_state=1)
# baseline = np.mean(cross_val_score(regression,df,y,scoring='r2',cv=crossvalidation))
baseline_all = cross_val_score(regression,df,y,scoring='r2',cv=crossvalidation)
baseline=np.mean(baseline_all)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [18]:
baseline

0.7190106820189471

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

In [22]:
from itertools import combinations
combinations = list(combinations(boston.feature_names, 2))
# Return 2-length subsequences of elements from the input iterable boston.feature_names.

In [24]:
print(boston.feature_names)
print(combinations)


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
[('CRIM', 'ZN'), ('CRIM', 'INDUS'), ('CRIM', 'CHAS'), ('CRIM', 'NOX'), ('CRIM', 'RM'), ('CRIM', 'AGE'), ('CRIM', 'DIS'), ('CRIM', 'RAD'), ('CRIM', 'TAX'), ('CRIM', 'PTRATIO'), ('CRIM', 'B'), ('CRIM', 'LSTAT'), ('ZN', 'INDUS'), ('ZN', 'CHAS'), ('ZN', 'NOX'), ('ZN', 'RM'), ('ZN', 'AGE'), ('ZN', 'DIS'), ('ZN', 'RAD'), ('ZN', 'TAX'), ('ZN', 'PTRATIO'), ('ZN', 'B'), ('ZN', 'LSTAT'), ('INDUS', 'CHAS'), ('INDUS', 'NOX'), ('INDUS', 'RM'), ('INDUS', 'AGE'), ('INDUS', 'DIS'), ('INDUS', 'RAD'), ('INDUS', 'TAX'), ('INDUS', 'PTRATIO'), ('INDUS', 'B'), ('INDUS', 'LSTAT'), ('CHAS', 'NOX'), ('CHAS', 'RM'), ('CHAS', 'AGE'), ('CHAS', 'DIS'), ('CHAS', 'RAD'), ('CHAS', 'TAX'), ('CHAS', 'PTRATIO'), ('CHAS', 'B'), ('CHAS', 'LSTAT'), ('NOX', 'RM'), ('NOX', 'AGE'), ('NOX', 'DIS'), ('NOX', 'RAD'), ('NOX', 'TAX'), ('NOX', 'PTRATIO'), ('NOX', 'B'), ('NOX', 'LSTAT'), ('RM', 'AGE'), ('RM', 'DIS'), ('RM', 'RAD'), ('RM', 'TAX'), (

In [31]:
len(combinations)

78

In [36]:
## code to find top 3 interactions by R^2 value here
rsquared_list = []
for combi in combinations:
    df['interaction']= df[combi[0]]*df[combi[1]]
#     crossvalidation = KFold(n_splits=10,shuffle=True,random_state=1)
    rsquared_list.append (np.mean (cross_val_score(regression,df,y,scoring='r2',cv=crossvalidation)))


In [39]:
result = list(zip(combinations,rsquared_list))
sorted(result, key = lambda x:x[1])

[(('CRIM', 'RM'), 0.7081164870054791),
 (('CHAS', 'RM'), 0.7086604865139804),
 (('AGE', 'LSTAT'), 0.713729880946088),
 (('CRIM', 'B'), 0.7146287027789207),
 (('RAD', 'LSTAT'), 0.7160169205618109),
 (('ZN', 'TAX'), 0.7165591526708505),
 (('TAX', 'LSTAT'), 0.7168545256468978),
 (('CHAS', 'PTRATIO'), 0.7168977502868781),
 (('AGE', 'RAD'), 0.7169145930312537),
 (('CHAS', 'DIS'), 0.7171246240196916),
 (('NOX', 'LSTAT'), 0.7172607859057891),
 (('RAD', 'B'), 0.7175471138330028),
 (('TAX', 'B'), 0.7175783335179269),
 (('AGE', 'TAX'), 0.7176729020685152),
 (('CHAS', 'AGE'), 0.717724117279856),
 (('CRIM', 'DIS'), 0.7177568432378603),
 (('DIS', 'RAD'), 0.7177953316377016),
 (('NOX', 'AGE'), 0.7178273549805465),
 (('ZN', 'B'), 0.7178806692926962),
 (('B', 'LSTAT'), 0.717893081877374),
 (('CRIM', 'INDUS'), 0.7180024566678991),
 (('INDUS', 'CHAS'), 0.7180330023698096),
 (('CRIM', 'ZN'), 0.718048657180336),
 (('RAD', 'PTRATIO'), 0.7180755401006279),
 (('INDUS', 'B'), 0.7180904055577036),
 (('ZN', 'CH

## Look at the top 3 interactions: "RM" as a confounding factor

The top three interactions seem to involve "RM", the number of rooms as a confounding variable for all of them. Let's have a look at interaction plots for all three of them. This exercise will involve:

- Splitting the data up in 3 groups: one for houses with a few rooms, one for houses with a "medium" amount of rooms, one for a high amount of rooms 
- Create a function `build_interaction_rm()`. This function takes an argument `varname` (which can be set equal to the column name as a string) and a column `description` (which describes the variable or varname, to be included on the x-axis of the plot). The function outputs a plot that uses "RM" as a confounding factor. Each plot should have three regression lines, one for each level of "RM"  

The data has been split into high, medium, and low number of rooms for you.

In [ ]:
rm = np.asarray(df[['RM']]).reshape(len(df[['RM']]))

In [ ]:
high_rm = all_data[rm > np.percentile(rm, 67)]
med_rm = all_data[(rm > np.percentile(rm, 33)) & (rm <= np.percentile(rm, 67))]
low_rm = all_data[rm <= np.percentile(rm, 33)]

Create `build_interaction_rm()`.

In [ ]:
def build_interaction_rm(varname, description):
    pass

Next, use `build_interaction_rm()` with the three variables that came out with the highest effect on $R^2$. 

In [ ]:
# first plot

In [ ]:
# second plot

In [ ]:
# third plot

## Build a final model including all three interactions at once

Use 10-fold cross-validation to build a model using all the above interactions. 

In [ ]:
# code here

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if all these interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.